In [6]:
from chi import server, context, lease
import os, time

context.version = "1.0" 
context.choose_project()
context.choose_site(default="CHI@TACC")

In [8]:
l = lease.get_lease(f"mlops-dday-project23")
l.show()

HTML(value='\n        <h2>Lease Details</h2>\n        <table>\n            <tr><th>Name</th><td>mlops-dday-pro…

Lease Details:
Name: mlops-dday-project23
ID: e0c6643e-15cd-4fd0-b6ff-1a056a470c09
Status: ACTIVE
Start Date: 2025-05-11 08:09:00
End Date: 2025-05-11 23:59:00
User ID: f9a642531ef78b49697f8beaba43bd4081bc2584478a0f280c0577547b859bc6
Project ID: d3c6e101843a4ba79e665ebf59b521a2

Node Reservations:
ID: 53230b33-4907-43c5-ab06-30a9117788f6, Status: active, Min: 1, Max: 1

Floating IP Reservations:

Network Reservations:

Events:


In [9]:
username = "project23"
s = server.Server(
    f"node-alstrain-{username}", 
    reservation_id=l.node_reservations[0]["id"],
    image_name="CC-Ubuntu24.04-hwe",
    key_name="id_rsa_project23"
)
s.submit(idempotent=True)

Waiting for server node-alstrain-project23's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,node-alstrain-project23
Id,96fa2597-30a6-4b10-9b61-2463ae2866ed
Status,ACTIVE
Image Name,CC-Ubuntu24.04-hwe
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.2.255 (v4) Type: fixed MAC: bc:97:e1:c4:d1:b0
Network Name,sharednet1
Created At,2025-05-11T08:41:29Z
Keypair,id_rsa_project23
Reservation Id,53230b33-4907-43c5-ab06-30a9117788f6
Host Id,9acf860df16fe3cd915f9522cd52cf171577a815ef5c486f67a143e3


In [10]:
s.associate_floating_ip()

In [11]:
s.refresh()
s.check_connectivity()
# ssh -i ~/.ssh/id_rsa_project23 cc@129.114.108.24

Checking connectivity to 129.114.108.24 port 22.


Connection successful


In [12]:
s.refresh()
s.show(type="widget")

Attribute,node-alstrain-project23
Id,96fa2597-30a6-4b10-9b61-2463ae2866ed
Status,ACTIVE
Image Name,CC-Ubuntu24.04-hwe
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.2.255 (v4) Type: fixed MAC: bc:97:e1:c4:d1:b0 IP: 129.114.108.24 (v4) Type: floating MAC: bc:97:e1:c4:d1:b0
Network Name,sharednet1
Created At,2025-05-11T08:41:29Z
Keypair,id_rsa_project23
Reservation Id,53230b33-4907-43c5-ab06-30a9117788f6
Host Id,9acf860df16fe3cd915f9522cd52cf171577a815ef5c486f67a143e3


## Setting up Docker

In [13]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 129.114.108.24: b'600d232c0389d2eee078e0fdc8322740'
  warnings.warn(


# Executing docker install script, commit: 53a22f61c0628e58e1d6680b49e82993d304b449


+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu noble stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install docker-ce docker-ce-cli containerd.io docker-compose-plugin docker-ce-rootless-extras docker-buildx-plugin >/dev/null

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
+ sh -c doc

Client: Docker Engine - Community
 Version:           28.1.1
 API version:       1.49
 Go version:        go1.23.8
 Git commit:        4eba377
 Built:             Fri Apr 18 09:52:14 2025
 OS/Arch:           linux/amd64
 Context:           default

Server: Docker Engine - Community
 Engine:
  Version:          28.1.1
  API version:      1.49 (minimum version 1.24)
  Go version:       go1.23.8
  Git commit:       01f442b
  Built:            Fri Apr 18 09:52:14 2025
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.7.27
  GitCommit:        05044ec0a9a75232cad458027ca83437aae3f4da
 runc:
  Version:          1.2.5
  GitCommit:        v1.2.5-0-g59923ef
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


To run Docker as a non-privileged user, consider setting up the
Docker daemon in rootless mode for your user:

    dockerd-rootless-setuptool.sh install

Visit https://docs.docker.com/go/rootless/ to learn about rootless mode.


T

<Result cmd='sudo groupadd -f docker; sudo usermod -aG docker $USER' exited=0>

## Pull and start Jupyter/PySpark container

In [14]:
s.execute("docker pull jupyter/pyspark-notebook:latest")

latest: Pulling from jupyter/pyspark-notebook
aece8493d397: Pulling fs layer
fd92c719666c: Pulling fs layer
088f11eb1e74: Pulling fs layer
4f4fb700ef54: Pulling fs layer
ef8373d600b0: Pulling fs layer
77e45ee945dc: Pulling fs layer
a30f89a0af6c: Pulling fs layer
dc42adc7eb73: Pulling fs layer
abaa8376a650: Pulling fs layer
aa099bb9e49a: Pulling fs layer
822c4cbcf6a6: Pulling fs layer
d25166dcdc7b: Pulling fs layer
964fc3e4ff9f: Pulling fs layer
2c4c69587ee4: Pulling fs layer
de2cdd875fa8: Pulling fs layer
75d33599f5f2: Pulling fs layer
4f4fb700ef54: Waiting
31973ea82470: Pulling fs layer
96ee7e4439c7: Pulling fs layer
1f9ad23c07ac: Pulling fs layer
ef8373d600b0: Waiting
d19266e0cb17: Pulling fs layer
9a165b6e9dc7: Pulling fs layer
5689442fd4e1: Pulling fs layer
dc42adc7eb73: Waiting
77e45ee945dc: Waiting
9a6a202f62a6: Pulling fs layer
a30f89a0af6c: Waiting
734ea0c3d94e: Pulling fs layer
a21a167f7127: Pulling fs layer
abaa8376a650: Waiting
467e20fcd668: Pulling fs layer
7024bb03412a: Pu

<Result cmd='docker pull jupyter/pyspark-notebook:latest' exited=0>

In [ ]:
s.execute("curl https://rclone.org/install.sh | sudo bash")
s.execute("sudo sed -i '/^#user_allow_other/s/^#//' /etc/fuse.conf")
s.execute("mkdir -p ~/.config/rclone")
s.execute('''cat > ~/.config/rclone/rclone.conf <<'EOF'
[chi_tacc]
type = swift
user_id = 59b49300deb6f11832f7764d9f2c3451ba51e578bb556d58e02bf4c64bd89a31
application_credential_id = aba2c756cb1649b6bd47c572f67a9528
application_credential_secret = HnUZC1Dvhnw1A5-19LiDQORiWURpL1IflqK3tk506NXJp8Tb0HSW4m0V9Ch4zEhwV9WR7tTxiAPTcTGHMi2SKw
auth = https://chi.tacc.chameleoncloud.org:5000/v3
region = CHI@TACC
EOF''')
s.execute("rclone lsd chi_tacc:")
s.execute("export RCLONE_CONTAINER=object-persist-project23")
s.execute("sudo mkdir -p /mnt/object")
s.execute("sudo chown -R cc /mnt/object")
s.execute("sudo chgrp -R cc /mnt/object")
s.execute("rclone mount chi_tacc:object-persist-project23 /mnt/object --read-only --allow-other --daemon")
s.execute("ls /mnt/object")

Mount the object store to the node so we can attach it to the Jupyterlabs container as a volume

    curl https://rclone.org/install.sh | sudo bash

    sudo sed -i '/^#user_allow_other/s/^#//' /etc/fuse.conf

    mkdir -p ~/.config/rclone

    # Write your rclone.conf
    cat > ~/.config/rclone/rclone.conf <<'EOF'
    [chi_tacc]
    type = swift
    user_id =   f9a642531ef78b49697f8beaba43bd4081bc2584478a0f280c0577547b859bc6
    application_credential_id = 6929d1f79f1146419f8836e5ef2da483
    application_credential_secret = gYN47v-JA0n61qBaMIVOBE-krxat-5kqQ04Q6hrDIGUxcx0F2m3zJ5CDFvZPskwdKPAYIt1OEzjoutpFduCsFQ
    auth = https://chi.tacc.chameleoncloud.org:5000/v3
    region = CHI@TACC
    EOF

    rclone lsd chi_tacc:

    export RCLONE_CONTAINER=object-persist-project23

    sudo mkdir -p /mnt/object
    sudo chown -R cc /mnt/object
    sudo chgrp -R cc /mnt/object

    rclone mount chi_tacc:object-persist-project23 /mnt/object --allow-other --daemon

    ls /mnt/object

Start the Jupyter container

s.execute("""
HOST_IP=$(curl --silent http://169.254.169.254/latest/meta-data/public-ipv4)
docker run -it --rm -p 8888:8888 \
--name pysparknb \
-v ~/my_workspace:/home/jovyan/work \
-v /home/cc/restaurant-recommender:/home/jovyan/restaurant-recommender \
-v /mnt/object:/home/jovyan/work/data \
-e RAY_ADDRESS=http://$HOST_IP:8265 \
-u 1000:1000 \
jupyter/pyspark-notebook
""")



Run this sommand to create the container

s.execute("docker compose -f docker-compose-ray-cpu.yaml up -d --build")
